![Alt Text](https://gradientflow.com/wp-content/uploads/2024/01/newsletter94-function-calling.jpeg)

# Creating LLM Functions
## OpenAI Function Schema
[OpenAI Function Calling Documentation](https://platform.openai.com/docs/guides/function-calling)

Step 1: Define the tools - weather data api from open weather map and search internet from duck duck go. 
Get your openweathermap api key from the [official website](https://home.openweathermap.org/). 

Step 2: Implement the search_internet and get_weather_info functions.

Step 3: Initialize the OpenAI client.

Step 4: Define the main conversation function run_conversation.

Step 5: Run the conversation with our tools.

## Import the required libraries

In [37]:
!pip install openai duckduckgo_search --quiet

## Define the tools/APIs

In [38]:
first_tools = [
    # Tool 1 - Get Weather Information
    { 
        "type": "function",
        "function": {
            "name": "get_weather_info",
            "description": "Get the current weather information for a specific location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location for which to fetch weather information, i.e., city name or coordinates.",
                    }
                },
                "required": ["location"],
            },
        },
    },
    # Tool 2 - Search Internet
    { 
        "type": "function",
        "function": {
            "name": "search_internet",
            "description": "Get internet search results for real-time information",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_query": {
                        "type": "string",
                        "description": "The query to search the web for",
                    }
                },
                "required": ["search_query"],
            },
        },
    }
]

## Implement the Functions
Define the search_internet and get_weather_info functions.

In [42]:
from duckduckgo_search import DDGS
import requests

def search_internet(search_query: str) -> list:
    results = DDGS().text(str(search_query), max_results=5)
    return results

def get_weather_info(location: str) -> dict:
    api_key = "add your open weather map api key"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return {
            "temperature": data["main"]["temp"],
            "description": data["weather"][0]["description"],
            "humidity": data["main"]["humidity"],
            "wind_speed": data["wind"]["speed"]
        }
    else:
        raise Exception(f"Failed to fetch weather information: {response.status_code}")

## Initialize the OpenAI Client
Initialize the OpenAI client before calling the run_conversation function.

In [43]:
from openai import OpenAI
import json

# Initialize the OpenAI client
client = OpenAI(api_key="add your OpenAI API key")

## Define the Main Conversation Function
Integrate the new functions into your conversation workflow.

In [44]:
import inspect

# Main conversation function
def run_conversation(prompt, tools, tool_choice = "auto"):
    
    messages = [{"role": "user", "content": prompt}]
    
    print("\nInitial Message: ", messages)
    
    # Send the conversation and available functions to the model
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice=tool_choice,
    )
    response_message = response.choices[0].message
    print("\nResponse Message: ", response_message)
    
    tool_calls = response_message.tool_calls
    print("\nTool Calls: ", tool_calls)
    
    # Check if the model wanted to call a function
    if tool_calls:
        
        # Call the functions
        available_functions = {
            "get_weather_info": get_weather_info,
            "search_internet": search_internet,
        } 
        # extend conversation with assistant's reply
        messages.append(response_message)
        
        # Call the function and add the response
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            
            # Get the function signature and call the function with given arguments
            sig = inspect.signature(function_to_call)
            call_args = {
                k: function_args.get(k, v.default)
                for k, v in sig.parameters.items()
                if k in function_args or v.default is not inspect.Parameter.empty
            }
            print(f"\nCalling {function_to_call} with arguments {call_args}")
            
            function_response = str(function_to_call(**call_args))
            
            print("\nFunction Response: ", function_response)

            # Put output into a tool message
            tool_message = {
                    "tool_call_id": tool_call.id, # Needed for Parallel Tool Calling
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            print("\nAppending Message: ", tool_message)
            
            # Extend conversation with function response
            messages.append(tool_message)  

        # Get a new response from the model where it can see the entire conversation including the function call outputs
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  

        print("\nLLM Response: ", second_response)

        print("\n---Formatted LLM Response---")
        print("\n",second_response.choices[0].message.content)
        
        return

## Run the Conversation
Now, use the run_conversation function with your tools.

In [46]:
prompt = "What's the weather like in Delhi? How about Hong Kong? Also, what's the current news in India?"

run_conversation(prompt, first_tools)


Initial Message:  [{'role': 'user', 'content': "What's the weather like in Delhi? How about Hong Kong? Also, what's the current news in India?"}]

Response Message:  ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_RBKG7w0f74y4VEkaz1qvJpKg', function=Function(arguments='{"location": "Delhi"}', name='get_weather_info'), type='function'), ChatCompletionMessageToolCall(id='call_WwUiRzHyRoIDIPJxeJCvbpqk', function=Function(arguments='{"location": "Hong Kong"}', name='get_weather_info'), type='function'), ChatCompletionMessageToolCall(id='call_95TFfxmvPfw6zfUsXs9gf8sS', function=Function(arguments='{"search_query": "current news in India"}', name='search_internet'), type='function')])

Tool Calls:  [ChatCompletionMessageToolCall(id='call_RBKG7w0f74y4VEkaz1qvJpKg', function=Function(arguments='{"location": "Delhi"}', name='get_weather_info'), type='function'), ChatCompletionMessageToolCall(id='call_WwU